In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [9]:
import random

import numpy as np
import pandas as pd

from tqdm import tqdm

from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN1
from src.models.gat import GAT

from src.models.train_model import train_with_loss
from src.models.train_model import random_splits

from src.models.reg import make_preg_loss
from src.models.reg import make_lap_loss
from src.models.intra_class_softmax import intra_class_distance

from src.models.reg import compute_a_hat

from src.models.evaluate_model import evaluate0
from src.models.evaluate_model import evaluate1

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from sklearn.manifold import TSNE

from src.models.evaluate_model import test

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0].to(device)

A_hat = compute_a_hat(data)

# data = random_splits(data, 50, 20)
data.reg_mask = torch.ones_like(data.train_mask, dtype=torch.bool)

L_cls = lambda x, y: F.cross_entropy(x, y, reduction='sum')
L_preg = lambda x, y: - (x * torch.log(y)).sum()


In [12]:
print('-------------------------------------------------------------')
print(f'train size: {data.train_mask.sum()}')
print(f'val size: {data.val_mask.sum()}')
print(f'test size: {data.test_mask.sum()}')
print('-------------------------------------------------------------')

metrics = []

for mu in range(9):
    torch.manual_seed(1)
    random.seed(1)

    mu = mu / 10
    loss_fn = make_preg_loss(L_cls, L_preg, mu, A_hat)
    
    model = GCN1(num_node_features=dataset.num_node_features,
        num_classes=dataset.num_classes) \
        .to(device)

    model = train_with_loss(model, data, loss_fn, num_epochs=200)

    acc = evaluate0(model, data)

    train_acc, val_acc, test_acc = evaluate1(model, data)
    icd = intra_class_distance(model(data), data)
    metrics.append({'mu': mu, 'train_acc': np.round(train_acc,4), 'val_acc': np.round(val_acc,4), 'test_acc': np.round(test_acc,4), 'intra_class_distance': np.round(icd, 4)})
    print(metrics[-1])
    # print(f'mu: {mu}, train_acc: {train_acc:.4f}, val_acc: {val_acc:.4f}, test_acc: {test_acc:.4f}')
    # print('-------------------------------------------------------------')


-------------------------------------------------------------
train size: 140
val size: 500
test size: 1000
-------------------------------------------------------------
{'mu': 0.0, 'train_acc': 0.9857, 'val_acc': 0.798, 'test_acc': 0.801, 'intra_class_distance': tensor(0.1213)}
{'mu': 0.1, 'train_acc': 0.9857, 'val_acc': 0.804, 'test_acc': 0.813, 'intra_class_distance': tensor(0.1193)}
{'mu': 0.2, 'train_acc': 0.9929, 'val_acc': 0.818, 'test_acc': 0.825, 'intra_class_distance': tensor(0.1195)}
{'mu': 0.3, 'train_acc': 0.9929, 'val_acc': 0.812, 'test_acc': 0.827, 'intra_class_distance': tensor(0.1186)}
{'mu': 0.4, 'train_acc': 0.9857, 'val_acc': 0.814, 'test_acc': 0.823, 'intra_class_distance': tensor(0.1182)}
{'mu': 0.5, 'train_acc': 0.9786, 'val_acc': 0.812, 'test_acc': 0.83, 'intra_class_distance': tensor(0.1196)}
{'mu': 0.6, 'train_acc': 0.9786, 'val_acc': 0.788, 'test_acc': 0.823, 'intra_class_distance': tensor(0.1195)}
{'mu': 0.7, 'train_acc': 0.9714, 'val_acc': 0.75, 'test_acc':